# Week 2 Day 2
We're going to build a simple Agent system for generating cold outreach emails:
    1. Agent workflow'
    2. Use of tools to call functions
    3. Agent collaboration via Tools and Handoffs

In [24]:
# Necessary Imports
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [25]:
# Load environment
load_dotenv(override=True)

True

# Step 1: Agent WorkFlow

In [26]:
# Create Agent Instructions
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [27]:
# Create Agents for each instruction

model_name="gpt-4o-mini"

# Professional Sales Agent
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model=model_name
)

# Engaging Sales Agent
sales_agent2 = Agent(
    "Engaging Sales Agent",
    instructions=instructions2,
    model=model_name
)

# Busy Sales Agent
sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instructions3,
    model=model_name
)


In [28]:
# Will use streaming API (.run_streamed())
# Run the instructions but call Runner.run_streamed(agent, instructions) in order to stream back results
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email.")

# Runner.run_streamed(agent, instructions) returns a coroutine object so we have to iterate through it 
# and call async in the for loop
async for event in result.stream_events():
    # Need to check that actual text was returned that we can print
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Streamline Your SOC 2 Compliance Process with AI

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out from ComplAI, where we specialize in simplifying the SOC 2 compliance process through our advanced AI-driven platform.

As you know, achieving and maintaining SOC 2 compliance can be complex and time-consuming. Our tool automates key aspects of the compliance lifecycle, offering tailored guidance that not only enhances your readiness for audits but also minimizes the risk of non-compliance. 

Here are a few ways ComplAI can add value to your organization:

- **Efficiency**: Reduce manual effort through automated workflows that streamline compliance tasks.
- **Real-time Monitoring**: Get insights into your compliance posture with continuous monitoring and reporting.
- **Expert Guidance**: Access a library of best practices and recommendations designed to simplify the auditing process.

I would love the opportunity to discuss 

In [29]:
# Going to use Asyncio for parallel agent calliing

message = "Send a cold sales email."

# Use trace to run agents asynchronously
with trace("Parallel Cold Emails"):
    # Will return a collection of results
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )

# Store outputs
outputs = [result.final_output for result in results]

# Iterate through outputs and print
for output in outputs:
    print(output + "\n\n")

Subject: Simplify Your SOC 2 Compliance with ComplAI

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m with ComplAI, where we specialize in helping organizations like yours streamline the SOC 2 compliance process.

Navigating compliance requirements can be complex and time-consuming, often diverting valuable resources from your core business activities. Our AI-powered SaaS tool is designed to simplify this process, offering features that ensure your organization meets all necessary compliance standards while preparing you for audits with ease.

With ComplAI, you can expect:

- Automated documentation and reporting
- Real-time compliance monitoring
- Customizable workflows tailored to your specific needs

I would love the opportunity to discuss how ComplAI can support your compliance efforts and enhance your operational efficiency. Are you available for a brief call next week?

Thank you for your time, and I look forward to the possibility of

In [30]:
# Create an agent that picks the best cold email
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. Imagine you are a customer and pick the one you are most likely to respond to. Do not give an explanation; reply with the selected email only.",
    model=model_name
)

In [31]:
# Define the message for Runner.run()
message = "Write a cold sales email"

# Run the agents asynchronously
with trace("Selection from sales people"):
    # Returns a list of coroutines
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )

# Gather the results into one list 
outputs = [result.final_output for result in results]

# Join the email strings from the 'outputs' list
emails = "Cold Sales Emails:\n\n".join(outputs)

# Run the sales picker to pick the best email
best_email = await Runner.run(sales_picker, emails)

# print out the best email
print(f"Best sales email:\n{best_email.final_output}")

Best sales email:
Subject: Don’t Let SOC2 Compliance Keep You Up at Night!

Hey [Recipient's Name],

Ever tried to juggle flaming swords while riding a unicycle? Now imagine doing that while trying to get SOC2 compliant. Sounds a bit crazy, right? 🤹‍♂️🔥

I’m reaching out from ComplAI — your soon-to-be favorite SaaS tool for mastering SOC2 compliance and making audits feel like a walk in the park (without any flaming swords!).

Our AI-driven platform takes the headache out of compliance, giving you back precious hours in your day (just think of all the coffee breaks you can enjoy!). ☕

Curious to see how we can turn that compliance process from chaos to calm? Let’s chat! I promise not to juggle anything on our call — unless you’re into that kind of thing.

Cheers,  
[Your Name]  
[Your Position] at ComplAI  
P.S. Did I mention we also have a killer sense of humor? But seriously, compliance doesn’t have to be boring!


# Step 2 and 3: Tools and Agent interactions
- Will now add tools to the mix

- Wrap function with the @function_tool decorator

In [39]:
# Define a function as a tool to send an email using SendGrid

@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects. """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    # Set the from email
    from_email = Email("twostrokes210business@gmail.com")
    # Set the to email
    to_email = To("twostrokes210business@gmail.com")
    # Set the email body content
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales Email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return{"status": "success"}


### This has automatically been converted into a tool, with the JSON boilerplate created

In [33]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects.', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000019558ED3D80>, strict_json_schema=True, is_enabled=True)

### And you can also convert an Agent into a tool

In [34]:
# Convert agent into a tool
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001955A4498A0>, strict_json_schema=True, is_enabled=True)

### So now we can gather all he tools together:
A tool fo reach of our 3 email-writing agents

And a tool for our function to send emails

In [35]:
# Create description
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001955A448360>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001955A22D6C0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'

## Now it's time for our Sales Manager - our planning agent!

In [40]:
# Create instructions for agent
instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. You never generate sales emails yourself; you always use the tools. You try all 3 sales_agent tools once before choosing the best one. You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."

# Create the sales agent with tools
sales_manager = Agent(
    name="Sales Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini"
)
# Create a messge for the agent to execute
message = "Send a cold sales email address to 'Dear CEO'"

# Define a trace for the agent
with trace("Sales_manager trace"):
    result = await Runner.run(sales_manager, message)
    print(result)

RunResult:
- Last agent: Agent(name="Sales Manager", ...)
- Final output (str):
    It seems there have been repeated issues trying to send the email. Unfortunately, I'm unable to complete that action at the moment. 
    
    If you have an email system, you can use the last generated email content to send it directly from your account. Would you like any additional assistance or modifications to the email?
- 19 new item(s)
- 8 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


### Remember to check the trace

https://platform.openai.com/traces

And then check your email!

### Handoffs represent a way an agent can delegate to an agent, passing control to it
Handoffs and Agents-as-tools are similar:
- In both cases, an Agent can collaborate with another Agent
- With tools, control passes back
- With handoffs, control passes acrosss

In [41]:
# Subject instructions
subject_instructions = "You can write a subject for a cold sales email. You are given a message and you need to write a subject for an email that is likely to get a response."

# HTML instructions
html_instructions = "You can convert a text email body to an HTML email body. You are given a text email body which might have some markdown and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

# Create subject_writer agent
subject_writer = Agent(
    name="Email subject writer",
    instructions=subject_instructions,
    model="gpt-4o-mini"
)
# Use the subject_writer agent to create a tool
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

# Create a html_converter agent
html_converter = Agent(
    name="HTML email body converter",
    instructions=html_instructions,
    model="gpt-4o-mini"
)
# Use the html_converter agent to create a tool
html_tool = html_converter.as_tool(tool_name="html_converter", tool_description="Convert a text email body to an HTML email body")

In [42]:
# Define function tool to send email
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with th egiven subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email  = Email("twostrokes210business@gmail.com")
    to_email = To("twostrokes210business@gmail.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)

    return{"status": "success"}


In [43]:
# Define tools list
tools = [subject_tool, html_tool, send_html_email]

In [44]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001955A448540>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001955A44AAC0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with th egiven subject and HTML body to 